In [1]:
import os
import pandas

In [2]:
data_batch = "train"
data_target = "rent"

In [3]:
# 获取文件夹下文件路径
def get_csv_files(folder_path):
    csv_files = []
    for filename in os.listdir(folder_path):
        # 组合完整路径
        filepath = os.path.join(folder_path, filename)
        # 过滤非文件类型和扩展名
        if os.path.isfile(filepath) and filename.lower().endswith(".csv"):
            csv_files.append(filepath)
    
    # 按编号排序
    csv_files.sort(key=lambda x: int(''.join(filter(str.isdigit, os.path.basename(x)))))
    return csv_files

In [4]:
folder_path = f"batch_result//客户反馈评级//{data_batch}_{data_target}"
out_path = f"客户反馈_{data_batch}_{data_target}.csv"
    
path_lst = get_csv_files(folder_path)
df_lst = [pandas.read_csv(path) for path in path_lst]

# 按行拼接
combined_df = pandas.concat(df_lst, ignore_index=True)

# 按列排序
# combined_df = combined_df.sort_values("feeling")

In [5]:
combined_df.to_csv(out_path, index=False)

In [6]:
# 处理房屋朝向
def extract_comment(text):
    henbuhao = bijiaobuhao = shizhong = bijiaohao = henhao = 0
    
    if "很不好" in text:
        henbuhao = 1
    if "比较不好" in text:
        bijiaobuhao = 1
    if "适中" in text:
        shizhong = 1
    if "比较好" in text:
        bijiaohao = 1
    if "很好" in text:
        henhao = 1
    
    return henbuhao, bijiaobuhao, shizhong, bijiaohao, henhao

In [7]:
# 检查缺失值
print(combined_df['客户反馈评级'].isnull().sum())
# 填充缺失值为空字符串
combined_df['客户反馈评级'] = combined_df['客户反馈评级'].fillna('')
# 应用提取函数并创建新列
combined_df[['很不好', '比较不好', '适中', '比较好', '很好']] = combined_df['客户反馈评级'].apply(
    lambda x: pandas.Series(extract_comment(x))
)

print(combined_df.columns)
print(combined_df.head())
print(f"数据维度：{combined_df.shape}")

0
Index(['ID', '客户反馈', '客户反馈评级', '很不好', '比较不好', '适中', '比较好', '很好'], dtype='object')
              ID                    客户反馈 客户反馈评级  很不好  比较不好  适中  比较好  很好
0  654646.481811          潮气重，仔细一看，房屋保养好     适中    0     0   1    0   0
1  665412.057415  服务响应中等，看起来，管线老化，消防设施齐全   比较不好    0     1   0    0   0
2  778222.820548  差不多这样，电梯新，总的来说，宽敞，性价比高    比较好    0     0   0    1   0
3  612084.974699       说起来也是，动静分离，社区功能齐全    比较好    0     0   0    1   0
4  994732.124864        服务响应中等，整体中等水平，宽敞     适中    0     0   1    0   0
数据维度：(98899, 8)


In [8]:
df_all = pandas.read_csv(f"结构化数据_{data_batch}_{data_target}.csv")

In [9]:
# 步骤1：检验关键列是否相同
if not df_all['ID'].equals(combined_df['ID']):
    # 如果列不完全相同，抛出异常（或根据需求调整）
    raise ValueError("关键列内容不相同，无法拼接！")

KeyError: 'ID'

In [ ]:
# 1. 检查基本属性
print("=== 列属性对比 ===")
print(f"Price列 | 长度: {len(df_all['Price'])} 类型: {df_all['Price'].dtype}")
print(f"ID列   | 长度: {len(combined_df['ID'])} 类型: {combined_df['ID'].dtype}")

# 2. 找出差异行
price_vals = df_all['Price'].reset_index(drop=True)
id_vals = combined_df['ID'].reset_index(drop=True)

diff_mask = price_vals != id_vals
diff_indices = diff_mask[diff_mask].index

# 3. 显示差异详情
print("\n=== 差异详情 ===")
print(f"共有 {len(diff_indices)} 处不同")
if len(diff_indices) > 0:
    sample_idx = diff_indices[:3]  # 显示前3个差异
    for idx in sample_idx:
        print(f"行 {idx}:")
        print(f"  Price值: {price_vals[idx]} ({type(price_vals[idx])})")
        print(f"  ID值  : {id_vals[idx]} ({type(id_vals[idx])})")

# 4. 特殊检查：NaN值
print("\n=== NaN检查 ===")
print(f"Price列NaN数量: {price_vals.isna().sum()}")
print(f"ID列NaN数量  : {id_vals.isna().sum()}")

In [10]:
# 步骤2：提取df2中需要拼接的列
df_subset = combined_df[['很不好', '比较不好', '适中', '比较好', '很好']].copy()

# 步骤3：横向拼接（注意：此处自动对齐索引）
result_df = pandas.concat([df_all, df_subset], axis=1)

In [11]:
result_df = result_df.drop("客户反馈", axis=1)

In [12]:
result_df.to_csv(f"结构化数据（带反馈）_{data_batch}_{data_target}.csv", index=False)